In [1]:
import os
import shutil
import sqlite3

import pandas as pd
import requests
import warnings
from dotenv import load_dotenv
from typing import Annotated, Dict, List, Optional, TypedDict, Union
from langchain_core.tools import BaseTool, InjectedToolCallId
from langgraph.graph.message import add_messages
from langgraph.prebuilt import InjectedState
from pydantic import BaseModel, Field
from datetime import datetime, date
from langchain_core.runnables import RunnableConfig
import uuid
import pytz

load_dotenv()
warnings.filterwarnings("ignore")

In [2]:
db = "../travel2.sqlite"
config = RunnableConfig(configurable={"passenger_id": "12345"})

### Tools

#### **Flight Tools**
1. Search Flights
   - **Input**: Departure airport, Arrival airport, start time, end time
   - **Output**: List of available flights with details

2. Fetch User's Flight details
    - **Input**: passenger_id
    - **Output**: List of flights booked by the user

3. Book Flight
    - **Input**: User ID, Flight ID, fare_conditions, Name, special_requests, meal_preferences
    - **Output**: Confirmation of booking

4. Cancel Flight
    - **Input**: User ID, Flight ID
    - **Output**: Confirmation of cancellation

5. Update Flight (update to next same flight)
    - **Input**: User ID, Flight ID, 
    - **Output**: Confirmation of update

In [3]:
class SearchFlights(BaseTool):
    name: str = "search_flights"
    description: str = """
    Searches for flight information using a third-party API.
    Use this tool when the user asks to find flights and you have all the necessary information:
    origin city, destination city, and departure date.
    Optionally takes number of adults, children, travel class, and if it's one-way.
    Do NOT call this tool if origin, destination, or departure date is missing.
    """

    def _run(
        self,
        departure_airport: str = None,
        arrival_airport: str = None,
        start_time: Optional[date | datetime] = None,
        end_time: Optional[date | datetime] = None,
        limit: int = 10,
    ) -> str:
        print(
            f"Executing search_flights with departure_airport={departure_airport}, "
            f"arrival_airport={arrival_airport}, start_time={start_time}, end_time={end_time}"
        )

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        query = "SELECT * FROM flights WHERE 1 = 1"
        params = []
        if departure_airport:
            query += " AND departure_airport = ?"
            params.append(departure_airport)

        if arrival_airport:
            query += " AND arrival_airport = ?"
            params.append(arrival_airport)

        if start_time:
            query += " AND DATE(scheduled_departure) >= DATE(?)"
            params.append(start_time)

        if end_time:
            query += " AND DATE(scheduled_departure) <= DATE(?)"
            params.append(end_time)
        query += " LIMIT ?"
        params.append(limit)
        cursor.execute(query, params)
        rows = cursor.fetchall()
        column_names = [column[0] for column in cursor.description]
        results = [dict(zip(column_names, row)) for row in rows]

        cursor.close()
        conn.close()

        return results

In [4]:
params = {
    "departure_airport": "LIS",
    "arrival_airport": "BOM",
    "start_time": "2025-06-30",
    "end_time": "2025-06-30",
}
SearchFlights().invoke(input=params)

Executing search_flights with departure_airport=LIS, arrival_airport=BOM, start_time=2025-06-30, end_time=2025-06-30


[{'flight_id': 14806,
  'flight_no': 'IB0093',
  'scheduled_departure': '2025-06-30 13:01:22.037749-04:00',
  'scheduled_arrival': '2025-06-30 13:41:22.037749-04:00',
  'departure_airport': 'LIS',
  'arrival_airport': 'BOM',
  'status': 'Scheduled',
  'aircraft_code': 'CR2',
  'actual_departure': None,
  'actual_arrival': None}]

In [ ]:
class BookFlight(BaseTool):
    name: str = "book_flight"
    description: str = """
    Book a flight using the provided flight details.
    Use this tool when the user has provided all necessary information for booking a flight.
    This includes the flight details, passenger information, and payment details.
    Do NOT call this tool if any required information is missing.
    The flight details should include the origin, destination, departure date, return date (if applicable), number of adults, number of children, and travel class.
    """

    def _run(
        self,
        config: RunnableConfig,
        flight_no: str,
        departure: date | datetime,
        book_ref: str,
        fare_conditions: Optional[str] = "None",
        meal_preference: Optional[str] = "None",
        special_assistance: Optional[str] = "None",
    ) -> str:
        print(
            f"Executing book_flight with flight_no={flight_no}, book_ref={book_ref}, fare_conditions={fare_conditions}, meal_preference={meal_preference}, special_assistance={special_assistance}"
        )

        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No passenger ID configured.")

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        # Get flight details
        query = "SELECT * FROM flights WHERE flight_no = ? AND DATE(scheduled_departure) = DATE(?)"
        cursor.execute(
            query,
            (
                flight_no,
                departure,
            ),
        )
        flight_details = cursor.fetchone()
        if not flight_details:
            raise ValueError(f"Flight with number {flight_no} not found.")

        # Get ticket
        query = (
            "SELECT * FROM ticket_flights WHERE flight_id = ? AND fare_conditions = ?"
        )
        cursor.execute(query, (flight_details[0], fare_conditions))
        ticket_details = cursor.fetchone()
        if not ticket_details:
            raise ValueError(f"Ticket for flight {flight_no} not found.")

        # Book the flight
        query = (
            "INSERT INTO tickets (ticket_no, book_ref, passenger_id) VALUES (?, ?, ?)"
        )
        params = [ticket_details[0], book_ref, passenger_id]

        cursor.execute(query, params)
        conn.commit()
        ticket_no = cursor.lastrowid

        # Generate boarding pass
        boarding_no = uuid.uuid4().hex[:8].upper()
        seat_no = f"{flight_details[1]}{uuid.uuid4().hex[:4].upper()}"

        query = "INSERT INTO boarding_passes (ticket_no, flight_id, boarding_no, seat_no) VALUES (?, ?, ?, ?)"
        params = [ticket_details[0], ticket_details[1], boarding_no, seat_no]
        cursor.execute(query, params)
        conn.commit()
        boarding_pass_id = cursor.lastrowid

        cursor.close()
        conn.close()

        return f"Flight booked successfully with ticket no: {ticket_no}"

In [6]:
conn = sqlite3.connect(db)
cursor = conn.cursor()

cursor.close()
conn.close()

In [7]:
params = {
    "flight_no": "DL0140",
    "book_ref": "REF123456",
    "fare_conditions": "Business",
    "departure": "2025-06-19",
    "meal_preference": "Vegetarian",
    "special_assistance": "Wheelchair",
}
BookFlight().invoke(input=params, config=config)

Executing book_flight with flight_no=DL0140, book_ref=REF123456, fare_conditions=Business, meal_preference=Vegetarian, special_assistance=Wheelchair


'Flight booked successfully with ticket no: 366738'

In [10]:
class CancelFlight(BaseTool):
    name: str = "cancel_flight"
    description: str = """
    Cancel a flight booking using the provided ticket number.
    Use this tool when the user wants to cancel a flight booking.
    The ticket number must be provided.
    Do NOT call this tool if the ticket number is missing.
    The tool will check if the ticket exists and if the current signed-in user is the owner of the ticket.
    """

    def _run(self, config: RunnableConfig, ticket_no: str) -> str:
        print(f"Executing cancel_flight with ticket_no={ticket_no}")

        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No passenger ID configured.")
        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        cursor.execute(
            "SELECT flight_id FROM ticket_flights WHERE ticket_no = ?", (ticket_no,)
        )
        existing_ticket = cursor.fetchone()
        if not existing_ticket:
            cursor.close()
            conn.close()
            return "No existing ticket found for the given ticket number."

        # Check the signed-in user actually has this ticket
        cursor.execute(
            "SELECT ticket_no FROM tickets WHERE ticket_no = ? AND passenger_id = ?",
            (ticket_no, passenger_id),
        )
        current_ticket = cursor.fetchone()
        if not current_ticket:
            cursor.close()
            conn.close()
            return f"Current signed-in passenger with ID {passenger_id} not the owner of ticket {ticket_no}"

        cursor.execute("DELETE FROM tickets WHERE ticket_no = ?", (ticket_no,))
        conn.commit()

        cursor.close()
        conn.close()

        return f"Flight with ticket ID {ticket_no} has been successfully canceled."

In [11]:
params = {
    "ticket_no": "0060005435215165",
}
CancelFlight().invoke(input=params, config=config)

Executing cancel_flight with ticket_no=0060005435215165


'No existing ticket found for the given ticket number.'

In [6]:
class UpdateFlight(BaseTool):
    name: str = "update_flight"
    description: str = """
    """

    def _run(
        self,
        config: RunnableConfig,
        ticket_no: str,
        new_flight_id: str,
    ) -> str:
        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No passenger ID configured.")

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        cursor.execute(
            "SELECT departure_airport, arrival_airport, scheduled_departure FROM flights WHERE flight_id = ?",
            (new_flight_id,),
        )
        new_flight = cursor.fetchone()
        if not new_flight:
            cursor.close()
            conn.close()
            return "Invalid new flight ID provided."
        column_names = [column[0] for column in cursor.description]
        new_flight_dict = dict(zip(column_names, new_flight))
        timezone = pytz.timezone("Etc/GMT-3")
        current_time = datetime.now(tz=timezone)
        departure_time = datetime.strptime(
            new_flight_dict["scheduled_departure"], "%Y-%m-%d %H:%M:%S.%f%z"
        )
        time_until = (departure_time - current_time).total_seconds()
        if time_until < (3 * 3600):
            return f"Not permitted to reschedule to a flight that is less than 3 hours from the current time. Selected flight is at {departure_time}."

        cursor.execute(
            "SELECT flight_id FROM ticket_flights WHERE ticket_no = ?", (ticket_no,)
        )
        current_flight = cursor.fetchone()
        if not current_flight:
            cursor.close()
            conn.close()
            return "No existing ticket found for the given ticket number."

        # Check the signed-in user actually has this ticket
        cursor.execute(
            "SELECT * FROM tickets WHERE ticket_no = ? AND passenger_id = ?",
            (ticket_no, passenger_id),
        )
        current_ticket = cursor.fetchone()
        if not current_ticket:
            cursor.close()
            conn.close()
            return f"Current signed-in passenger with ID {passenger_id} not the owner of ticket {ticket_no}"

        # In a real application, you'd likely add additional checks here to enforce business logic,
        # like "does the new departure airport match the current ticket", etc.
        # While it's best to try to be *proactive* in 'type-hinting' policies to the LLM
        # it's inevitably going to get things wrong, so you **also** need to ensure your
        # API enforces valid behavior
        cursor.execute(
            "UPDATE ticket_flights SET flight_id = ? WHERE ticket_no = ?",
            (new_flight_id, ticket_no),
        )
        conn.commit()

        cursor.close()
        conn.close()
        return "Ticket successfully updated to new flight."

In [ ]:
# params = {"ticket_no": "9880005434068393", "new_flight_id": ""}
# UpdateFlight().invoke(input=params, config=config)

#### **Hotel Tools**
1. Search Hotels
   - **Input**: location, name, price_tier
   - **Output**: List of available hotels with details

2. Fetch User's hotel details
    - **Input**: passenger_id
    - **Output**: List of hotels booked by the user

3. Book Hotel
    - **Input**: hotel_id, passenger_id, check_in_date, check_out_date, room_type
    - **Output**: Confirmation of booking

4. Cancel Hotel Booking
    - **Input**: passenger_id, hotel_id
    - **Output**: Confirmation of cancellation

5. update Hotel Booking (update check-in/check-out dates only)
    - **Input**: passenger_id, hotel_id, check_in_date, check_out_date
    - **Output**: Confirmation of update

In [8]:
class SearchHotel(BaseTool):
    name: str = "search_hotel"
    description: str = """
    Search for hotels based on location, name, price tier, check-in date, and check-out date.

    Args:
        location (Optional[str]): The location of the hotel. Defaults to None.
        name (Optional[str]): The name of the hotel. Defaults to None.
        price_tier (Optional[str]): The price tier of the hotel. Defaults to None. Examples: Midscale, Upper Midscale, Upscale, Luxury

    Returns:
        list[dict]: A list of hotel dictionaries matching the search criteria.
    """

    def _run(
        self,
        location: str = None,
        name: str | None = None,
        price_tier: str | None = None,
    ) -> list[dict]:
        print(
            f"Executing search_hotel with location={location}, price_tier={price_tier}"
        )

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        query = "SELECT * FROM hotels WHERE 1=1"
        params = []

        if location:
            query += " AND location LIKE ?"
            params.append(f"%{location}%")
        if name:
            query += " AND name LIKE ?"
            params.append(f"%{name}%")

        cursor.execute(query, params)
        results = cursor.fetchall()

        conn.close()

        return [
            dict(zip([column[0] for column in cursor.description], row))
            for row in results
        ]

In [ ]:
params = {
    "location": "Basel",
    "price_tier": "Luxury",
}
SearchHotel().invoke(input=params)

Executing search_hotel with location=Basel, price_tier=Luxury


[{'id': 1,
  'name': 'Hilton Basel',
  'location': 'Basel',
  'price_tier': 'Luxury',
  'checkin_date': '2024-04-22',
  'checkout_date': '2024-04-20',
  'booked': 0},
 {'id': 3,
  'name': 'Hyatt Regency Basel',
  'location': 'Basel',
  'price_tier': 'Upper Upscale',
  'checkin_date': '2024-04-02',
  'checkout_date': '2024-04-20',
  'booked': 0},
 {'id': 8,
  'name': 'Holiday Inn Basel',
  'location': 'Basel',
  'price_tier': 'Upper Midscale',
  'checkin_date': '2024-04-24',
  'checkout_date': '2024-04-09',
  'booked': 0}]

In [18]:
class BookHotel(BaseTool):
    name: str = "book_hotel"
    description: str = """
    Book a hotel by its ID.

    Args:
        hotel_id (int): The ID of the hotel to book.

    Returns:
        str: A message indicating whether the hotel was successfully booked or not.
    """

    def _run(
        self,
        config: RunnableConfig,
        hotel_id: str,
        check_in_date: date,
        check_out_date: date,
        room_type: Optional[str] = None,
        num_guests: int = 1,
    ) -> str:
        print(
            f"Executing book_hotel with hotel_id={hotel_id}, check_in_date={check_in_date}, check_out_date={check_out_date}, num_guests={num_guests}"
        )

        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No passenger ID configured.")

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        # Get hotel details
        query = "SELECT * FROM hotels WHERE id = ?"
        cursor.execute(query, (hotel_id,))
        hotel_details = cursor.fetchone()
        if not hotel_details:
            raise ValueError(f"Hotel with ID {hotel_id} not found.")

        # Book the hotel
        booking_id = uuid.uuid4().hex[:8].upper()
        query = (
            "INSERT INTO hotel_bookings (booking_id, hotel_id, passenger_id, check_in_date, check_out_date, room_type, num_guests) "
            "VALUES (?, ?, ?, ?, ?, ?, ?)"
        )
        params = [
            booking_id,
            hotel_id,
            passenger_id,
            check_in_date,
            check_out_date,
            room_type,
            num_guests,
        ]

        cursor.execute(query, params)
        conn.commit()
        booking_id = cursor.lastrowid

        cursor.close()
        conn.close()

        return f"Hotel booked successfully with booking ID: {booking_id}"

In [ ]:
params = {
    "hotel_id": "2",
    "check_in_date": "2024-01-01",
    "check_out_date": "2024-01-05",
    "room_type": "Deluxe",
    "num_guests": 2,
}
BookHotel().invoke(input=params, config=config)

Executing book_hotel with hotel_id=2, check_in_date=2024-01-01, check_out_date=2024-01-05, num_guests=2


'Hotel booked successfully with booking ID: 2'

In [25]:
class UpdateHotelBooking(BaseTool):
    name: str = "update_hotel_booking"
    description: str = """
    Update a hotel's check-in and check-out dates by its ID.

    Args:
        hotel_id (int): The ID of the hotel to update.
        checkin_date (Optional[Union[datetime, date]]): The new check-in date of the hotel. Defaults to None.
        checkout_date (Optional[Union[datetime, date]]): The new check-out date of the hotel. Defaults to None.

    Returns:
        str: A message indicating whether the hotel was successfully updated or not.
    """

    def _run(
        self,
        config: RunnableConfig,
        new_check_in_date: date,
        new_check_out_date: date,
    ) -> str:
        print(
            f"Executing update_hotel_booking with new_check_in_date={new_check_in_date}, new_check_out_date={new_check_out_date}"
        )

        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No booking ID configured.")

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        # Get Booking ID
        query = "SELECT * FROM hotel_bookings WHERE passenger_id = ?"
        cursor.execute(query, (passenger_id,))
        booking_details = cursor.fetchone()
        print(booking_details)

        if not booking_details:
            cursor.close()
            conn.close()
            return "No existing booking found for the given passenger ID."
        booking_id = booking_details[0]

        # Update the hotel booking
        query = "UPDATE hotel_bookings SET check_in_date = ?, check_out_date = ? WHERE booking_id = ?"
        params = [new_check_in_date, new_check_out_date, booking_id]

        cursor.execute(query, params)
        conn.commit()

        cursor.close()
        conn.close()

        return f"Hotel booking with ID {booking_id} successfully updated."

In [26]:
params = {
    "new_check_in_date": "2026-01-01",
    "new_check_out_date": "2026-01-05",
}
config = RunnableConfig(configurable={"passenger_id": "12345"})
UpdateHotelBooking().invoke(input=params, config=config)

Executing update_hotel_booking with new_check_in_date=2026-01-01, new_check_out_date=2026-01-05
('D36E807E', '2', 12345, '2024-01-01', '2024-01-05', 'Deluxe', 2)


'Hotel booking with ID D36E807E successfully updated.'

In [31]:
class CancelHotelBooking(BaseTool):
    name: str = "cancel_hotel_booking"
    description: str = """
    Cancel a hotel by its ID.

    Args:
        hotel_id (int): The ID of the hotel to cancel.

    Returns:
        str: A message indicating whether the hotel was successfully cancelled or not.
    """

    def _run(self, config: RunnableConfig, hotel_id: str) -> str:
        print(f"Executing cancel_hotel_booking with hotel_id={hotel_id}")

        configuration = config.get("configurable", {})
        passenger_id = configuration.get("passenger_id", None)
        if not passenger_id:
            raise ValueError("No passenger ID configured.")

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        # Check if the booking exists
        query = "SELECT * FROM hotel_bookings WHERE passenger_id = ?"
        cursor.execute(query, (passenger_id,))
        existing_booking = cursor.fetchone()
        if not existing_booking:
            cursor.close()
            conn.close()
            return "No existing booking found for the given booking ID."

        # Cancel the hotel booking
        booking_id = existing_booking[0]
        query = "DELETE FROM hotel_bookings WHERE booking_id = ?"
        cursor.execute(query, (booking_id,))
        conn.commit()

        cursor.close()
        conn.close()

        return f"Hotel booking with ID {booking_id} has been successfully canceled."

In [32]:
params = {"hotel_id": "2"}
config = RunnableConfig(configurable={"passenger_id": "12345"})
CancelHotelBooking().invoke(input=params, config=config)

Executing cancel_hotel_booking with hotel_id=2


'Hotel booking with ID D36E807E has been successfully canceled.'

#### **Car Rentals**
1. Search Car Rentals
   - **Input**: location, name, price_tier, start_time, end_time
   - **Output**: List of available car rentals with details

2. Fetch User's Car Rental details
    - **Input**: passenger_id
    - **Output**: List of car rentals booked by the user

3. Book Car Rental
    - **Input**: rental_id, passenger_id, start_time, end_time
    - **Output**: Confirmation of booking

4. Cancel Car Rental Booking
    - **Input**: rental_id, passenger_id
    - **Output**: Confirmation of cancellation

5. update Hotel Booking (update start time/end time only)
    - **Input**: rental_id, passenger_id, start_time, end_time
    - **Output**: Confirmation of update

In [ ]:
class SearchCarRental(BaseTool):
    name: str = "search_car_rental"
    description: str = """
    Search for car rentals based on location, name, price tier, start date, and end date.

    Args:
        location (Optional[str]): The location of the car rental. Defaults to None.
        name (Optional[str]): The name of the car rental company. Defaults to None.
        price_tier (Optional[str]): The price tier of the car rental. Defaults to None.
        start_date (Optional[Union[datetime, date]]): The start date of the car rental. Defaults to None.
        end_date (Optional[Union[datetime, date]]): The end date of the car rental. Defaults to None.

    Returns:
        list[dict]: A list of car rental dictionaries matching the search criteria.
    """

    def _run(
        self,
        location: str = None,
        name: str | None = None,
        price_tier: str | None = None,
        start_date: Optional[date | datetime] = None,
        end_date: Optional[date | datetime] = None,
    ) -> list[dict]:
        print(
            f"Executing search_hotel with location={location}, price_tier={price_tier}"
        )

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        query = "SELECT * FROM car_rentals WHERE 1=1"
        params = []

        if location:
            query += " AND location LIKE ?"
            params.append(f"%{location}%")
        if name:
            query += " AND name LIKE ?"
            params.append(f"%{name}%")

        cursor.execute(query, params)
        results = cursor.fetchall()

        conn.close()

        return [
            dict(zip([column[0] for column in cursor.description], row))
            for row in results
        ]

In [15]:
params = {
    "location": "Basel",
    "price_tier": "Luxury",
    "start_date": "2024-01-01",
    "end_date": "2024-01-05",
}
SearchCarRental().invoke(input=params)

Executing search_hotel with location=Basel, price_tier=Luxury


[{'id': 1,
  'name': 'Europcar',
  'location': 'Basel',
  'price_tier': 'Economy',
  'start_date': '2024-04-14',
  'end_date': '2024-04-11',
  'booked': 0},
 {'id': 2,
  'name': 'Avis',
  'location': 'Basel',
  'price_tier': 'Luxury',
  'start_date': '2024-04-10',
  'end_date': '2024-04-20',
  'booked': 0},
 {'id': 7,
  'name': 'Enterprise',
  'location': 'Basel',
  'price_tier': 'Premium',
  'start_date': '2024-04-22',
  'end_date': '2024-04-20',
  'booked': 0},
 {'id': 9,
  'name': 'Thrifty',
  'location': 'Basel',
  'price_tier': 'Midsize',
  'start_date': '2024-04-17',
  'end_date': '2024-04-26',
  'booked': 0}]

In [16]:
class BookCarRental(BaseTool):
    name: str = "book_car_rental"
    description: str = """
    Book a car rental by its ID.

    Args:
        rental_id (int): The ID of the car rental to book.

    Returns:
        str: A message indicating whether the car rental was successfully booked or not.
    """

    def _run(
        self,
        config: RunnableConfig,
        rental_id: str,
        start_date: date,
        end_date: date,
    ) -> str:
        print(
            f"Executing book_car_rental with rental_id={rental_id}, start_date={start_date}, end_date={end_date}"
        )

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        cursor.execute("UPDATE car_rentals SET booked = 1 WHERE id = ?", (rental_id,))
        conn.commit()

        if cursor.rowcount > 0:
            conn.close()
            return f"Car rental {rental_id} successfully booked."
        else:
            conn.close()
            return f"No car rental found with ID {rental_id}."

In [19]:
params = {
    "rental_id": "1",
    "start_date": "2024-01-01",
    "end_date": "2024-01-05",
}
BookCarRental().invoke(input=params)

Executing book_car_rental with rental_id=1, start_date=2024-01-01, end_date=2024-01-05


'Car rental 1 successfully booked.'

In [14]:
class UpdateCarRental(BaseTool):
    name: str = "update_car_rental"
    description: str = """
    Update a car rental's start and end dates by its ID.

    Args:
        rental_id (int): The ID of the car rental to update.
        start_date (Optional[Union[datetime, date]]): The new start date of the car rental. Defaults to None.
        end_date (Optional[Union[datetime, date]]): The new end date of the car rental. Defaults to None.

    Returns:
        str: A message indicating whether the car rental was successfully updated or not.
    """

    def _run(
        self,
        config: RunnableConfig,
        rental_id: str,
        start_date: date,
        end_date: date,
    ) -> str:
        print(
            f"Executing book_car_rental with rental_id={rental_id}, start_date={start_date}, end_date={end_date}"
        )

        conn = sqlite3.connect(db)
        cursor = conn.cursor()

        if start_date:
            cursor.execute(
                "UPDATE car_rentals SET start_date = ? WHERE id = ?",
                (start_date, rental_id),
            )
        if end_date:
            cursor.execute(
                "UPDATE car_rentals SET end_date = ? WHERE id = ?",
                (end_date, rental_id),
            )

        conn.commit()

        if cursor.rowcount > 0:
            conn.close()
            return f"Car rental {rental_id} successfully updated."
        else:
            conn.close()
            return f"No car rental found with ID {rental_id}."

In [24]:
params = {
    "rental_id": "1",
    "start_date": "2024-01-01",
    "end_date": "2024-01-05",
}
UpdateCarRental().invoke(input=params)

Executing book_car_rental with rental_id=1, start_date=2024-01-01, end_date=2024-01-05


'Car rental 1 successfully updated.'

In [22]:
class CancelCarRental(BaseTool):
    name: str = "cancel_car_rental"
    description: str = """
    Cancel a car rental by its ID.

    Args:
        rental_id (int): The ID of the car rental to cancel.

    Returns:
        str: A message indicating whether the car rental was successfully cancelled or not.
    """

    def _run(
        self,
        config: RunnableConfig,
        rental_id: str,
    ) -> str:
        print(f"Executing cancel_car_rental with rental_id={rental_id}")

        conn = sqlite3.connect(db)

        cursor = conn.cursor()

        cursor.execute("UPDATE car_rentals SET booked = 0 WHERE id = ?", (rental_id,))
        conn.commit()

        if cursor.rowcount > 0:
            conn.close()
            return f"Car rental {rental_id} successfully cancelled."
        else:
            conn.close()
            return f"No car rental found with ID {rental_id}."

In [25]:
params = {
    "rental_id": "1",
}
CancelCarRental().invoke(input=params)

Executing cancel_car_rental with rental_id=1


'Car rental 1 successfully cancelled.'